In [ ]:
import numpy as np 
import pandas as pd 
# import os 
import scipy 
from scipy . optimize import curve_fit
from scipy . interpolate import interp1d
from astropy.io import fits #allows python to interpurt fits data files 
from glob import glob # Unix style pathname pattern expansion
from matplotlib import pyplot as plt 
from astropy.wcs import WCS
from astropy.stats import sigma_clip
from photutils import DAOStarFinder
from astropy.stats import mad_std
from photutils import aperture_photometry, CircularAperture, CircularAnnulus
%matplotlib inline

# --- plot parameters --- 
plt.rcParams["figure.figsize"] = (15,10)
plt.rc('font', family = 'serif', serif = 'cmr10') 
plt.rcParams.update({'font.size': 22})

In [ ]:
# --- Data Import of Lab Data --- 

NNSer_bias_data = [fits.getdata(image) for image in glob('lab_data/Ser/bias/B120308*')]
print(np.shape(NNSer_bias_data))

NNSer_flat_clear_data = [fits.getdata(image) for image in glob('lab_data/Ser/flats/clear/F120308*')]
print(np.shape(NNSer_flat_clear_data))

NNSer_object_clear_data = [fits.getdata(image) for image in glob('lab_data/Ser/object/clear/O120308*')]
print(np.shape(NNSer_object_clear_data))

# --- Clipping Overscan --- 


master_bias_NNSer = np.average(NNSer_bias_data, axis = 0)
master_bias_NNSer = np.delete(master_bias_NNSer, np.s_[0:50], axis = 1)
print(np.shape(master_bias_NNSer))
print(master_bias_NNSer.mean())

master_flat_NNSer = np.average(NNSer_flat_clear_data, axis = 0)
master_flat_NNSer = np.delete(master_flat_NNSer, np.s_[0:50], axis = 1)
print(np.shape(master_flat_NNSer))

master_flat_NNSer_Norm =  master_flat_NNSer/master_flat_NNSer.mean()
print(np.shape(master_flat_NNSer_Norm))
print(np.mean(master_flat_NNSer_Norm))

In [ ]:
# --- Plots of Master Flat and Bias --- 

# --- Plotting Master Bias --- 
bmax = master_bias_NNSer.mean() + master_bias_NNSer.std()
bmin = master_bias_NNSer.mean() - master_bias_NNSer.std()

plt.imshow(master_bias_NNSer, cmap = 'magma', vmin = bmin , vmax = bmax)
# plt.title("Master Bias for NNSer")
plt.grid()
plt.colorbar()
plt.show()
plt.savefig("images/Ser/NNSer_Master_Bias.svg")

# --- Plotting Master Flat --- 
bmax = master_flat_NNSer.mean() + master_flat_NNSer.std()
bmin = master_flat_NNSer.mean() - master_flat_NNSer.std()

plt.imshow(master_flat_NNSer, cmap = 'magma', vmin = bmin , vmax = bmax)
# plt.title("Master Flat for NNSer")
plt.grid()
plt.colorbar()
plt.show()
plt.savefig("images/Ser/NNSer_Master_Flat.svg")

# --- Plotting Master Flat Normalised --- 
bmax = master_flat_NNSer_Norm.mean() + master_flat_NNSer_Norm.std()
bmin = master_flat_NNSer_Norm.mean() - master_flat_NNSer_Norm.std()

plt.imshow(master_flat_NNSer_Norm, cmap = 'magma', vmin = vmin , vmax = vmax)
# plt.title("Normalised Master Flat for NNSer")
plt.grid()
plt.colorbar()
plt.show()
plt.savefig("images/Ser/NNSer_Master_Flat_Norm.svg")

In [ ]:
# --- Saving Master Files --- 

fits.PrimaryHDU(master_bias_NNSer).writeto('Ser_Redlab_data\Ser_Master_Bias', overwrite = True)
fits.PrimaryHDU(master_flat_NNSer).writeto('Ser_Redlab_data\Ser_Master_Flat', overwrite = True)
fits.PrimaryHDU(master_flat_NNSer_Norm).writeto('Ser_Redlab_data\Ser_Master_Flat_Normalised', overwrite = True)

In [ ]:
# --- Reducing Object Images and Producing a master file --- 

object_B = glob('lab_data\Ser\object\clear\O120308*')
# print(object_B)
object_B_pixels = NNSer_object_clear_data

mb_clipped = fits.getdata('Ser_Redlab_data\Ser_Master_Bias')

for file in object_B:
    object_BB = fits.open(file) # stores fit file 
    object_B_pixels = object_BB[0].data # access of pixel data 
    object_B_subtracted = np.delete(object_B_pixels, np.s_[0:50], axis=1) - mb_clipped #
    object_B_reduced =  object_B_subtracted/master_flat_NNSer_Norm #
    object_B_pixels = object_B_reduced
    HDU = fits.PrimaryHDU(object_B_reduced)
    name = 'Ser_Red' + file.split('/')[-1] #"Ser\object_reduced\object_reduced_" +
    # print(newfilename)
    HDU.writeto(name, overwrite = True)

In [ ]:
reduced_object = glob('Ser_Redlab_data/Ser/object/clear/O120308*')
reduced_object_data = [fits.getdata(image) for image in glob('Ser_Redlab_data/Ser/object/clear/O120308*')]
print(np.shape(reduced_object_data))

# --- Combine --- 
combined_reduced_object_data = np.average(reduced_object_data, axis = 0)

In [ ]:
# --- Looking at the reduced image output before proceeding further 

test_print = glob('Ser_Redlab_data/Ser/object/clear/O120308*')
sources_array = []

for i in range(0, len(test_print) - 1):

    # --- Data Import --- 
    data = reduced_object_data[i]
    print(np.shape(data))
    print(data.mean())
    std = (np.std(data))
    vmin = data.mean() - data.std()
    vmax = data.mean() + data.std()

    # --- Sources --- 
    std = mad_std(data)
    daofind = DAOStarFinder(fwhm = 6., threshold = 5.*std)
    mst_sources = daofind(data)
    print(len(mst_sources['id']))
    sources_array = np.append(sources_array, len(mst_sources['id']))

    #--- Plot --- 
    plt.imshow(np.delete(data, np.s_[0:50], axis=1), cmap = 'magma', vmin = vmin, vmax = vmax) 
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
# --- Test Plot --- 

bmax = combined_reduced_object_data.mean() + combined_reduced_object_data.std()
bmin = combined_reduced_object_data.mean() - combined_reduced_object_data.std()

plt.imshow(np.delete(combined_reduced_object_data, np.s_[0:50], axis=1), cmap = 'magma', vmin = bmin , vmax = bmax)
plt.colorbar()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# --- Saving the Combined Image --- 
fits.PrimaryHDU(combined_reduced_object_data).writeto('master_images\master_image_NNSer', overwrite = True)

In [ ]:
NNSer_master_image = fits.open('master_images/master_image_NNSer')
image_data = NNSer_master_image[0].data
image_data = np.delete(image_data, np.s_[0:50], axis=1)

# print(image_data)
# print(type(image_data))

mst_std = mad_std(image_data)
daofind = DAOStarFinder(fwhm = 6., threshold = 5.*mst_std)
mst_sources = daofind(image_data)

print(type(mst_sources))

len(mst_sources['id'])

In [ ]:
for file in glob('Ser_Redlab_data/Ser/object/clear/O120308*'):
    image_data = fits.open(file)
    image_data = image_data[0].data
    print(image_data.mean())

for image in glob('Ser_Redlab_data/Ser/object/clear/O120308*'):
#     print(image)
    image = fits.getdata(image)
    print(image.mean())
   
    bkg_sigma = mad_std(image)
    daofind = DAOStarFinder(fwhm=6., threshold= 5*bkg_sigma)
    sources = daofind(image)
    # Get the positions of sources in the field from the table above.
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

    # Set up aperture and annulus
    aperture = CircularAperture(positions, r=5.)
    annulus_aperture = CircularAnnulus(positions, r_in=10., r_out=15.)

    # Make a list of apertures
    apers = [aperture, annulus_aperture]

    # And run aperture photometry
    phot_table = aperture_photometry(image, apers)


    # We calculate the mean counts in each pixel in the background annulus, and then multiply by the area
    # in the aperture to get the total background counts within each aperture

    bkg_mean = (phot_table['aperture_sum_1'])/(annulus_aperture.area)
    bkg_sum = bkg_mean * aperture.area

    # Now we get the final table of background subtracted counts within each aperture
    final_sum = phot_table['aperture_sum_0'] - bkg_sum


    plt.imshow(image, vmin=np.mean(image)-np.std(image), vmax=np.mean(image)+np.std(image), cmap = 'magma')
#     print(np.mean(image))
    plt.annotate('D', (mst_sources['xcentroid'][34], mst_sources['ycentroid'][34]), color = 'blue')
    plt.annotate('NN', (mst_sources['xcentroid'][51], mst_sources['ycentroid'][51]), color = 'blue')
    plt.annotate(''' 1' ''', (mst_sources['xcentroid'][73], mst_sources['ycentroid'][73]), color = 'blue')
    plt.annotate('A', (mst_sources['xcentroid'][52], mst_sources['ycentroid'][52]), color = 'blue')
    plt.annotate('C', (mst_sources['xcentroid'][65], mst_sources['ycentroid'][65]), color = 'blue')
    plt.annotate('B', (mst_sources['xcentroid'][66], mst_sources['ycentroid'][66]), color = 'blue')
#     plt.xlim(200,1000)
#     plt.ylim(1100,0)
    
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    
    plt.colorbar()
    plt.show()


In [ ]:


# --- Plotting Objects --- 

vmin = image_data.mean() - image_data.std()
vmax = image_data.mean() + image_data.std()

plt.imshow(image_data, vmin = vmin, vmax = vmax, cmap = 'magma')
# plt.scatter(M91mst_sources['xcentroid'], M91mst_sources['ycentroid'], alpha = 0.3, color = 'green')
plt.gca().invert_yaxis()
# plt.xlim(2000, 750)
# plt.ylim(500, 1750)
# plt.gca().invert_xaxis()
plt.colorbar()
plt.savefig("images/Ser/unmarked_photometry_NNSer.svg")

for i in range(0, len(mst_sources)): 
    plt.annotate(i, (mst_sources['xcentroid'][i], mst_sources['ycentroid'][i]), color = 'blue')

In [ ]:
reduced_object = glob('Ser_Redlab_data/Ser/object/clear/O120308*')
r_object = np.sort(reduced_object)
# print(r_object)

r_object_data = [fits.getdata(image) for image in r_object] 
r_object_avg = np.average(r_object_data, axis = 0)

In [ ]:
# --- Plotting Objects --- 

vmin = image_data.mean() - image_data.std()
vmax = image_data.mean() + image_data.std()

plt.imshow(image_data, vmin = vmin, vmax = vmax, cmap = 'magma')
# plt.scatter(M91mst_sources['xcentroid'], M91mst_sources['ycentroid'], alpha = 0.3, color = 'green')
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.colorbar()
plt.annotate('D', (mst_sources['xcentroid'][34], mst_sources['ycentroid'][34]), color = 'blue')
plt.annotate('NN', (mst_sources['xcentroid'][51], mst_sources['ycentroid'][51]), color = 'blue')
plt.annotate(''' 1' ''', (mst_sources['xcentroid'][73], mst_sources['ycentroid'][73]), color = 'blue')
plt.annotate('A', (mst_sources['xcentroid'][52], mst_sources['ycentroid'][52]), color = 'blue')
plt.annotate('C', (mst_sources['xcentroid'][65], mst_sources['ycentroid'][65]), color = 'blue')
plt.annotate('B', (mst_sources['xcentroid'][66], mst_sources['ycentroid'][66]), color = 'blue')
# plt.title('Comparison Stars Labelled with NNSer')
plt.savefig("images/Ser/marked_photometry_NNSer.svg")

In [ ]:
# --- Plotting Objects --- 

vmin = image_data.mean() - image_data.std()
vmax = image_data.mean() + image_data.std()

plt.imshow(image_data, vmin = vmin, vmax = vmax, cmap = 'magma')
# plt.scatter(M91mst_sources['xcentroid'], M91mst_sources['ycentroid'], alpha = 0.3, color = 'green')
plt.gca().invert_yaxis()
plt.xlim(2000, 750)
plt.ylim(500, 1750)
# plt.gca().invert_xaxis()
plt.colorbar()
plt.annotate('D', (mst_sources['xcentroid'][34], mst_sources['ycentroid'][34]), color = 'blue')
plt.annotate('NN', (mst_sources['xcentroid'][51], mst_sources['ycentroid'][51]), color = 'blue')
plt.annotate(''' 1' ''', (mst_sources['xcentroid'][73], mst_sources['ycentroid'][73]), color = 'blue')
plt.annotate('A', (mst_sources['xcentroid'][52], mst_sources['ycentroid'][52]), color = 'blue')
plt.annotate('C', (mst_sources['xcentroid'][65], mst_sources['ycentroid'][65]), color = 'blue')
plt.annotate('B', (mst_sources['xcentroid'][66], mst_sources['ycentroid'][66]), color = 'blue')
# plt.title('Comparison Stars Labelled with NNSer')
plt.savefig("images/Ser/marked_photometry_NNSer_zoom.svg")

In [ ]:
# --- Photo-Metry --- 

image_data = r_object_avg

mst_std = mad_std(image_data)
daofind = DAOStarFinder(fwhm = 6., threshold = 5.*mst_std)
mst_sources = daofind(image_data)

coords = np.transpose((mst_sources['xcentroid'], mst_sources['ycentroid'])) # transposes so the array is line correctly with projected image. 
# print(coords, np.shape(coords), type(coords))

In [ ]:
def SN(source_counts, sky_counts, pixels_source, pixels_sky, R):
    return source_counts/np.sqrt(source_counts + pixels_source *(1 + pixels_source / pixels_sky )*(sky_counts + R**2) )

def SNsig(SN):
    return 2.5*np.log(1 + 1/SN) 

def mean_err(std, n):
    return np.sqrt(std)/np.sqrt(n) 

In [ ]:
x = [fits.getdata(image) for image in glob('Ser_Redlab_data/Ser/object/clear/O120308*')]
print(np.shape(x))

x = [(fits.open(file))[0].data for file in glob('Ser_Redlab_data/Ser/object/clear/O120308*')]
print(np.shape(x))